In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('Master101618v4.csv')

/Users/epetenko/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (35,36,45,67,85,87,90,95,96) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,﻿id,Timestamp,Date_inputted,County,Town,Town_county,File_name,Case_report_no,Incident_date,Incident_time,...,Subject_3_charges_st,Subject_3_force_nature,Subject_3_injured,Subject_3_hosp,Subject_3_killed,Subject_3_photo,Is_second,Is_third,Subject_2_force_nature_new,Subject_3_force_nature_new
0,2115,2/23/18 1:33,2/21/18,Atlantic County,Absecon,"Absecon, Atlantic County",ABSECON-UOF-2012-2016,I-2016-028167,11/6/16,10:26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
1,1755,2/20/18 3:09,2/21/18,Atlantic County,Absecon,"Absecon, Atlantic County",ABSECON-UOF-2012-2016,2012-015242,6/27/12,18:26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
2,2092,2/22/18 5:21,2/21/18,Atlantic County,Absecon,"Absecon, Atlantic County",ABSECON-UOF-2012-2016,I-2015-001527,1/14/15,2:11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
3,2108,2/22/18 7:26,2/21/18,Atlantic County,Absecon,"Absecon, Atlantic County",ABSECON-UOF-2012-2016,I-2016-003974,2/14/16,4:34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
4,2109,2/22/18 23:47,2/21/18,Atlantic County,Absecon,"Absecon, Atlantic County",ABSECON-UOF-2012-2016,I-2016-011235,5/9/16,15:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70561 entries, 0 to 70560
Data columns (total 99 columns):
﻿id                           70561 non-null int64
Timestamp                     70430 non-null object
Date_inputted                 70401 non-null object
County                        70561 non-null object
Town                          70561 non-null object
Town_county                   70561 non-null object
File_name                     70517 non-null object
Case_report_no                70557 non-null object
Incident_date                 70561 non-null object
Incident_time                 70561 non-null object
Location_city                 22498 non-null object
Location_detail               70559 non-null object
Incident_type                 69938 non-null object
Incident_EDP                  5488 non-null object
Officer_1_first               70561 non-null object
Officer_1_last                70561 non-null object
officerid                     70561 non-null object
Officer_1

### Prep columns: Incident key and years

In [6]:
df['Incident_key'] = df['Town_county'] + ', ' + df.Case_report_no + ', ' + df.Incident_date

In [7]:
df['Year'] = df.Incident_date.str.split(pat='/', expand=True)[2]

df.Year = df.Year.replace(to_replace='12', value='2012')
df.Year = df.Year.replace(to_replace='13', value='2013')
df.Year = df.Year.replace(to_replace='14', value='2014')
df.Year = df.Year.replace(to_replace='15', value='2015')
df.Year = df.Year.replace(to_replace='16', value='2016')

df.Year = df.Year.replace(to_replace='12 0:00', value='2012')
df.Year = df.Year.replace(to_replace='13 0:00', value='2013')
df.Year = df.Year.replace(to_replace='14 0:00', value='2014')
df.Year = df.Year.replace(to_replace='15 0:00', value='2015')
df.Year = df.Year.replace(to_replace='16 0:00', value='2016')


## Incident totals:

#### Total uses of force 2012-2016:

In [8]:
df.Incident_key.count()

70557

#### Total uses of force by year:

In [9]:
df.pivot_table(index='Year', values='Incident_key', aggfunc=len)

Year
2012    13314
2013    14002
2014    14199
2015    14129
2016    14777
XXXX       21
Name: Incident_key, dtype: int64

#### Incidents 2012-2016 and by year:

In [10]:
print 'Unique incidents, 2012-2016:', df['Incident_key'].nunique()

Unique incidents, 2012-2016: 43637


In [11]:
by_year = df.pivot_table(index='Year', values='Incident_key', aggfunc=lambda x: len(x.unique()))

by_year

Year
2012    8751
2013    8794
2014    8782
2015    8501
2016    8703
XXXX      17
Name: Incident_key, dtype: int64

##  Arrest rates

#### Read in arrest dataset

In [12]:
spreadsheet_id = "1rd5K7MKdFnoFxIaAVF9LVT9J_EZA1HmRRxKm_ZF_ub8"
to_pre = pd.read_csv("https://docs.google.com/spreadsheets/d/" + spreadsheet_id + "/export?format=csv")

In [13]:
to_pre.head()

,AGENCY,STNAME,ORI,Invensis_county,Invensis_town,Invensis_town_county,TOTAL_ARRESTS_2012,TOTAL_ARRESTS_2013,TOTAL_ARRESTS_2014,TOTAL_ARRESTS_2015,...,Margin of Error; Not Hispanic or Latino: - Asian alone,Estimate; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone,Margin of Error; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone,Estimate; Hispanic or Latino:,Margin of Error; Hispanic or Latino:,CENSUS_WHITE_PERCENT,CENSUS_BLACK_PERCENT,CENSUS_AMERICAN_INDIAN_PERCENT,CENSUS_ASIAN_PERCENT,CENSUS_HISPANIC_PERCENT
0,ABERDEEN TOWNSHIP,N J,NJ01330,Monmouth County,Aberdeen,"Aberdeen, Monmouth County",617.0,575.0,584.0,626.0,...,323.0,8.0,13.0,1756.0,285.0,70.407829,10.764269,0.049202,5.980757,9.599825
1,ABSECON,N J,NJ00101,Atlantic County,Absecon,"Absecon, Atlantic County",355.0,384.0,291.0,228.0,...,210.0,0.0,17.0,886.0,249.0,74.117089,7.110690,0.118511,5.842617,10.500119
2,ALLENDALE,N J,NJ00201,Bergen County,Allendale,"Allendale, Bergen County",82.0,84.0,103.0,105.0,...,207.0,0.0,17.0,134.0,78.0,83.744643,1.108320,0.000000,11.216196,1.980198
3,ALLENHURST,N J,NJ01301,Monmouth County,Allenhurst,"Allenhurst, Monmouth County",141.0,152.0,135.0,116.0,...,11.0,5.0,10.0,15.0,13.0,94.354839,0.000000,0.000000,1.612903,3.024194
4,ALLENTOWN,N J,NJ01302,Monmouth County,Allentown,"Allentown, Monmouth County",20.0,32.0,64.0,37.0,...,13.0,0.0,12.0,56.0,42.0,90.158730,2.751323,0.000000,0.846561,2.962963


#### Total arrests, and by year:

In [14]:
ar = to_pre[['TOTAL_ARRESTS_2012', 'TOTAL_ARRESTS_2013', 'TOTAL_ARRESTS_2014', 
       'TOTAL_ARRESTS_2015', 'TOTAL_ARRESTS_2016']].sum()

ar

TOTAL_ARRESTS_2012    292861.0
TOTAL_ARRESTS_2013    278472.0
TOTAL_ARRESTS_2014    270359.0
TOTAL_ARRESTS_2015    257809.0
TOTAL_ARRESTS_2016    262737.0
dtype: float64

In [15]:
tot_arrests = ar.sum()

#### Non-edp incidents by year

In [16]:
all_UOF_data_no_blank_charges = df[(df['Incident_EDP'] != 'EDP') & (df['Subject_1_charges'] != 'EDP/MEDICAL')  | (df['Subject_1_charges'] != 'BLANK')]

year_nedp = all_UOF_data_no_blank_charges.pivot_table(index='Year', values='Incident_key', aggfunc=lambda x: len(x.unique()))
year_nedp

Year
2012    8447
2013    8458
2014    8387
2015    8060
2016    8099
XXXX      16
Name: Incident_key, dtype: int64

### Incidents per arrest:

In [17]:
print "Incidents per 1,000 arrests", (float(df[df.Incident_EDP != 'EDP'].Incident_key.nunique()) / tot_arrests ) * 1000

Incidents per 1,000 arrests 29.8831775358


In [56]:
print "Incidents per 1,000 arrests 2012:", (8447 / 284206.0 ) * 1000
print "Incidents per 1,000 arrests 2013:", (8458 / 270867.0 ) * 1000
print "Incidents per 1,000 arrests 2014:", (8387 / 261662.0 ) * 1000
print "Incidents per 1,000 arrests 2015:", (8060 / 250233.0 ) * 1000
print "Incidents per 1,000 arrests 2016:", (8099 / 255830.0) * 1000

Incidents per 1,000 arrests 2012: 29.7213992667
Incidents per 1,000 arrests 2013: 31.2256568722
Incidents per 1,000 arrests 2014: 32.0528009417
Incidents per 1,000 arrests 2015: 32.2099802984
Incidents per 1,000 arrests 2016: 31.6577414689


## Incidents by race:

#### Percentage of subjects in incidents by race:

In [19]:
race_piv = pd.DataFrame(df.pivot_table(index='Subject_1_race', values='Incident_key', aggfunc=lambda x: len(x.unique())))

race_piv['Percent of total'] = (race_piv['Incident_key'] / race_piv.Incident_key.sum(axis=0)) * 100

race_piv

,Incident_key,Percent of total
Subject_1_race,,
asian,273,0.613994
asian/pacific islander,117,0.263140
black,17457,39.261858
blank,1,0.002249
canine,4,0.008996
hispanic,4447,10.001574
illegible,18,0.040483
mixed,4,0.008996
native american,17,0.038234


#### Racial breakdown excluding EDPs:

In [20]:
race_nedp = pd.DataFrame(df[df.Incident_EDP != 'EDP'].pivot_table(index='Subject_1_race', values='Incident_key', aggfunc=lambda x: len(x.unique())))

race_nedp['Percent of total'] = (race_nedp['Incident_key'] / race_nedp.Incident_key.sum(axis=0)) * 100

race_nedp

,Incident_key,Percent of total
Subject_1_race,,
asian,238,0.573757
asian/pacific islander,106,0.255539
black,16793,40.483595
blank,1,0.002411
canine,4,0.009643
hispanic,4256,10.260119
illegible,15,0.036161
mixed,4,0.009643
native american,15,0.036161


#### Percent of arrests:

In [21]:
wh_arrests = to_pre[[u'TOTAL_WHITE_ARRESTS_2012', u'TOTAL_WHITE_ARRESTS_2013',
       u'TOTAL_WHITE_ARRESTS_2014', u'TOTAL_WHITE_ARRESTS_2015',
       u'TOTAL_WHITE_ARRESTS_2016']].sum().sum()

bl_arrests = to_pre[[u'TOTAL_BLACK_ARRESTS_2012',
       u'TOTAL_BLACK_ARRESTS_2013', u'TOTAL_BLACK_ARRESTS_2014',
       u'TOTAL_BLACK_ARRESTS_2015', u'TOTAL_BLACK_ARRESTS_2016']].sum().sum()

hi_arrests = 0

api_arrests = to_pre[[u'TOTAL_API_ARRESTS_2013', u'TOTAL_API_ARRESTS_2014',
       u'TOTAL_API_ARRESTS_2015', u'TOTAL_API_ARRESTS_2016']].sum().sum()

aian_arrests = to_pre[[  u'TOTAL_AIAN_ARRESTS_2012', u'TOTAL_AIAN_ARRESTS_2013',
       u'TOTAL_AIAN_ARRESTS_2014', u'TOTAL_AIAN_ARRESTS_2015',
       u'TOTAL_AIAN_ARRESTS_2016']].sum().sum()

In [22]:
print 'White people percent of arrests: ', wh_arrests / tot_arrests * 100
print 'Black people percent of arrests: ', bl_arrests / tot_arrests * 100
print 'Hispanic people percent of arrests: ', hi_arrests / tot_arrests * 100
print 'API people percent of arrests: ', api_arrests / tot_arrests * 100
print 'AIAN people percent of arrests: ', aian_arrests / tot_arrests * 100

White people percent of arrests:  54.7369108775
Black people percent of arrests:  34.816236223
Hispanic people percent of arrests:  0.0
API people percent of arrests:  1.04108092712
AIAN people percent of arrests:  0.149606750069


#### Percent of population:

In [23]:
# Read in Census summary because some of the pop data appeared to be messed up in the arrests file (prob. NJSP)
pop_df = pd.read_csv('Data_sources/Race_pop/ACS_16_5YR_B03002_with_ann.csv', skiprows=1)

In [24]:
tot_pop = float(pop_df['Estimate; Total:'].sum())
wh_pop = float(pop_df['Estimate; Not Hispanic or Latino: - White alone'].sum())
bl_pop = float(pop_df['Estimate; Not Hispanic or Latino: - Black or African American alone'].sum())
hi_pop = float( pop_df['Estimate; Hispanic or Latino:'].sum())
api_pop = float( pop_df['Estimate; Not Hispanic or Latino: - Asian alone'].sum() + pop_df['Estimate; Not Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone'].sum())
aian_pop = float( pop_df['Estimate; Not Hispanic or Latino: - American Indian and Alaska Native alone'].sum())

In [25]:
print 'White people percent of population:', (wh_pop / tot_pop) * 100
print 'Black people percent of population:', (bl_pop / tot_pop) * 100
print 'Hispanic people percent of population:', (hi_pop / tot_pop) * 100
print 'API people percent of population:', (api_pop / tot_pop) * 100
print 'AIAN people percent of population:', (aian_pop / tot_pop) * 100

White people percent of population: 56.6949239613
Black people percent of population: 12.7185642552
Hispanic people percent of population: 19.2904434725
API people percent of population: 9.15246511227
AIAN people percent of population: 0.106657472147


#### White-black odds ratios:

In [26]:
print "Black - white population odds ratio:" , ((race_piv.loc[['black'], 
                                                              ['Incident_key']].Incident_key.sum() / bl_pop) / (race_piv.loc[['white'],
                                                                                                                             ['Incident_key']].Incident_key.sum() / wh_pop))

Black - white population odds ratio: 3.6962531717


In [27]:
# excluded EDPs
print "Black - white arrests odds ratio:" , ((race_nedp.loc[['black'], 
                                                              ['Incident_key']].Incident_key.sum() / bl_arrests) / (race_nedp.loc[['white'],
                                                                                                                             ['Incident_key']].Incident_key.sum() / wh_arrests))

Black - white arrests odds ratio: 1.38531784635


## Officer statistics

#### Total number of unique officers:

In [28]:
df.officerid.nunique()

19231

#### Incidents per officer, on average:

In [29]:
float(df.Incident_key.count()) / df.officerid.nunique()

3.6689199729603246

#### Officer racial breakdown

In [30]:
offrace = pd.DataFrame(df.pivot_table(index='Officer_1_race', values='Incident_key', aggfunc=len))
offrace['Percent'] = (offrace.Incident_key / offrace.Incident_key.sum(axis=0)) * 100
offrace

,Incident_key,Percent
Officer_1_race,,
asian,531,0.752540
asian/pacific islander,121,0.171483
black,4467,6.330693
black/white,2,0.002834
hispanic,3980,5.640510
illegible,23,0.032596
mixed,27,0.038265
native american,7,0.009920
not listed,19224,27.244512


## Top officers


In [31]:
#basic list of top officers
top_off_start = pd.DataFrame(df.officerid.value_counts().head(26))
top_off_start = top_off_start.rename(columns={'officerid':'Number of incidents for this officer'})
top_off_start.head()

,Number of incidents for this officer
ScottMSendrickAtlanticCity,62
JamesDayVineland,59
JosephDixonMillville,58
RyanPDuffyPaterson,56
PatrickJoyceLongBranch,52


In [32]:
#got rid of MIs cause they were throwing first names off
df.Officer_1_first = df.Officer_1_first.str.split(pat=' ', expand=True)[0]

#then created pivot table of info you're looking for
all_off = pd.DataFrame(df.pivot_table(index=['officerid', 'Officer_1_first', 'Officer_1_last', 'Officer_1_rank', 
                               'Officer_1_race', 'Town_county'], values='Incident_key', aggfunc=len)).reset_index()

all_off = all_off.rename(columns={'Incident_key': 'Rows with this data'})

In [33]:
#merge em
top_off_all = top_off_start.merge(all_off, how='left', left_index=True, right_on='officerid')

In [34]:
#Pivot table of department totals
town_county_piv = pd.DataFrame(df.pivot_table(index='Town_county', values='Incident_key', aggfunc=lambda x: len(x.unique())))
town_county_piv = town_county_piv.rename(columns={'Incident_key': 'Department_incidents'})

In [35]:
top_off = top_off_all.merge(town_county_piv, how='left', left_on='Town_county', right_index=True)

top_off['Percent of department'] = (top_off['Number of incidents for this officer'] / top_off['Department_incidents']) * 100

In [36]:
top_off.head()

,Number of incidents for this officer,officerid,Officer_1_first,Officer_1_last,Officer_1_rank,Officer_1_race,Town_county,Rows with this data,Department_incidents,Percent of department
21214,62,ScottMSendrickAtlanticCity,Scott,Sendrick,Ofc.,not listed,"Atlantic City, Atlantic County",4,1671,3.710353
21215,62,ScottMSendrickAtlanticCity,Scott,Sendrick,Ofc.,white,"Atlantic City, Atlantic County",58,1671,3.710353
9518,59,JamesDayVineland,James,Day,Ofc.,white,"Vineland, Cumberland County",59,916,6.441048
12197,58,JosephDixonMillville,Joseph,Dixon,Ofc.,not listed,"Millville, Cumberland County",58,454,12.775330
20804,56,RyanPDuffyPaterson,Ryan,Duffy,Ofc.,not listed,"Paterson, Passaic County",56,1467,3.817314


In [37]:
# top_off.to_csv('Data_slices/Top_officers_in_replication.csv')

## Types of force/reasons for force

#### Nature of force used

In [38]:
#list of categories
force = ['compliance hold', 'hands/fist',  'leg strikes', 
         'tackle/wrestle to the ground',  'chemical agent', 
         'strike/use of baton or other object', 'deadly']

In [39]:
#function that runs through and searches for categories
def get_string_data(list_name, column_name):
    new_piv = []
    for n in list_name:
        this_type_count = df[df[column_name].str.contains(n)].Incident_key.count()
        this_type_unique = df[df[column_name].str.contains(n)].Incident_key.nunique()
        this_type_percent_count = (float(this_type_count) / df.Incident_key.count()) * 100
        this_type_percent_unique = (float(this_type_unique) / df.Incident_key.nunique()) * 100
        this_list = n, this_type_count, this_type_unique, this_type_percent_count, this_type_percent_unique
        new_piv.append(this_list)
    return pd.DataFrame(new_piv, columns=['Force', 'Number of rows', 'Number of unique incidents', 'Percent of rows', 'Percent of incidents'])

In [40]:
#creating the final pivot
force_piv = get_string_data(force, 'Subject_1_force_nature_new')

In [41]:
force_piv

,Force,Number of rows,Number of unique incidents,Percent of rows,Percent of incidents
0,compliance hold,56990,35142,80.771575,80.532576
1,hands/fist,19826,13809,28.099267,31.645164
2,leg strikes,3008,2446,4.263220,5.605335
3,tackle/wrestle to the ground,4347,3428,6.160976,7.855719
4,chemical agent,4966,4438,7.038281,10.170268
5,strike/use of baton or other object,1176,1028,1.666738,2.355799
6,deadly,345,250,0.488966,0.572908


### Deadly force

In [42]:
print 'Number of rows: ', df[df.Firearms_discharged == 'Intentional'].Incident_key.count()

Number of rows:  366


In [43]:
print 'Number of incidents: ', df[df.Firearms_discharged == 'Intentional'].Incident_key.nunique()

Number of incidents:  266


### Subject actions

In [44]:
actions = ['resisted police officer control', 'physical threat/attack officer or another -- self-defense',
          'threatened/attacked officer or another with a blunt object', 
          'threatened/attacked officer or another with a knife/cutting object',
          'threatened officer or another with firearm',
           'fire firearm at officer or another',
           'threatened/ attacked officer or another with a motor vehicle']

In [45]:
actions_piv = get_string_data(actions, 'Subject_1_actions_OG')

In [46]:
actions_piv

,Force,Number of rows,Number of unique incidents,Percent of rows,Percent of incidents
0,resisted police officer control,63300,38810,89.714699,88.938286
1,physical threat/attack officer or another -- s...,24979,15403,35.402582,35.298027
2,threatened/attacked officer or another with a ...,692,458,0.980767,1.049568
3,threatened/attacked officer or another with a ...,563,363,0.797936,0.831863
4,threatened officer or another with firearm,521,333,0.738410,0.763114
5,fire firearm at officer or another,99,53,0.140312,0.121457
6,threatened/ attacked officer or another with a...,473,322,0.670380,0.737906


## Injured and hospitalized

### Subjects injuried

In [47]:
subj_inj = pd.DataFrame(df.pivot_table(index='Subject_1_injured', values='Incident_key', aggfunc=lambda x: len(x.unique())))

In [48]:
print 'Subjects percent injured: ',  float(subj_inj.loc[['yes'], ['Incident_key']].Incident_key.sum() )/  subj_inj.Incident_key.sum(axis=0)

Subjects percent injured:  0.208359451957


### Subjects hospitalized

In [49]:
subj_hosp = pd.DataFrame(df.pivot_table(index='Subject_1_hosp', values='Incident_key', aggfunc=lambda x: len(x.unique())))

print 'Subjects percent hospitalized: ',  float(subj_hosp.loc[['yes'], ['Incident_key']].Incident_key.sum() )/ subj_hosp.Incident_key.sum(axis=0)

Subjects percent hospitalized:  0.0994907774131


In [50]:
subj_hosp

,Incident_key
Subject_1_hosp,
No,1
no,10798
not listed,28986
redacted,4
yes,4396


### Officers injured

In [51]:
off_inj = pd.DataFrame(df.pivot_table(index='Officer_1_injured', values='Incident_key', aggfunc=lambda x: len(x.unique())))

print 'Officers percent injured: ',  float(off_inj.loc[['yes'], ['Incident_key']].Incident_key.sum() )/ off_inj.Incident_key.sum(axis=0)

Officers percent injured:  0.11850294004


### Officers hospitalized

In [52]:
off_hosp = pd.DataFrame(df.pivot_table(index='Officer_1_hosp', values='Incident_key', aggfunc=lambda x: len(x.unique())))

print 'Officers percent hospitalized: ',  float(off_hosp.loc[['yes'], ['Incident_key']].Incident_key.sum() )/ off_hosp.Incident_key.sum(axis=0)

Officers percent hospitalized:  0.0240048685931


## Top towns

#### read in arrests data and get total arrests

In [53]:
to_pre['Tot_arrests'] = to_pre.TOTAL_ARRESTS_2012 + to_pre.TOTAL_ARRESTS_2013 + to_pre.TOTAL_ARRESTS_2014  + to_pre.TOTAL_ARRESTS_2015 + to_pre.TOTAL_ARRESTS_2016
to = to_pre[[ u'Invensis_town_county', u'Tot_arrests']]
to_piv = pd.DataFrame(to.pivot_table(index='Invensis_town_county', values=['Tot_arrests'], aggfunc=sum))

#### Merge with non-EDP numbers

In [54]:
t_a = pd.DataFrame(df[df.Incident_EDP != 'EDP'].groupby('Town_county').Incident_key.nunique()).merge(to_piv, 
                                                                     left_index=True, 
                                                                     right_index=True, 
                                                                     how='left')

t_a['Incidents_per_1000_arrests'] = (t_a.Incident_key / t_a.Tot_arrests) * 1000

In [55]:
t_a[t_a.index != 'NJSP, State Police'].sort_values(by='Incidents_per_1000_arrests', ascending=False).head(25)

,Incident_key,Tot_arrests,Incidents_per_1000_arrests
Town_county,,,
"Maplewood, Essex County",172,1549.0,111.039380
"Atlantic City, Atlantic County",1653,14973.0,110.398718
"Pine Beach, Ocean County",10,99.0,101.010101
"Woodland Park, Passaic County",52,527.0,98.671727
"Pleasantville, Atlantic County",161,1661.0,96.929561
"Wenonah, Gloucester County",12,127.0,94.488189
"Sea Girt, Monmouth County",50,578.0,86.505190
"Sea Isle City, Cape May County",73,933.0,78.242229
"Voorhees, Camden County",172,2207.0,77.933847
